In [1]:
all_match_ids = []

In [ ]:
import requests
import time

class RiotApi(object):
    def __init__(self, api_key: str, region="americas"):
        self.RIOT_API_KEY = api_key
        self.HEADER = {'X-Riot-Token': self.RIOT_API_KEY}
        self.REGION = region
        self.BASE_URL = ".api.riotgames.com/"

    def get_puuid_by_riot_id(self, game_name: str, tag_line: str) -> dict:
        """Get PUUID by Riot ID (gameName and tagLine)
        @param game_name: Riot ID gameName
        @param tag_line: Riot ID tagLine
        @return: json object of account data
        """
        url = f"https://{self.REGION}{self.BASE_URL}riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}"
        request = requests.get(url, headers=self.HEADER)
        if request.status_code == 200:
            return request.json()
        else:
            print(f"Error: Received status code {request.status_code} for PUUID request. Response: {request.text}")
            return {}

    def get_summoner_by_puuid(self, puuid: str) -> dict:
        """Get Summoner Data by PUUID
        @param puuid: Player Universal Unique Identifier
        @return: json object of summoner data
        """
        self.REGION = "na1"
        url = f"https://{self.REGION}{self.BASE_URL}lol/summoner/v4/summoners/by-puuid/{puuid}"
        request = requests.get(url, headers=self.HEADER)
        if request.status_code == 200:
            return request.json()
        else:
            print(f"Error: Received status code {request.status_code} for summoner request. Response: {request.text}")
            return {}
        

    def get_match_ids(self, puuid, count=20):
        self.REGION = "americas"
        match_ids = []
        start = 0
        
        while True:
            url = f"https://{self.REGION}{self.BASE_URL}/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count={count}"
            request = requests.get(url, headers=self.HEADER)
            if request.status_code == 200:
                batch_ids = request.json()
                if not batch_ids:
                    break
                match_ids.extend(batch_ids)
                start += count
                time.sleep(1.2)  # Adding a delay to avoid rate limits
            else:
                print(f"Error: Received status code {request.status_code} for match ID request. Response: {request.text}")
                break

        return match_ids
        
    def match_details(self, match_id):
        self.REGION = "americas"
        url = f"https://{self.REGION}{self.BASE_URL}/lol/match/v5/matches/{match_id}"
        request = requests.get(url, headers=self.HEADER)
        if request.status_code == 200:
            return request.json()
        else:
            print(f"Error: Received status code {request.status_code} for summoner request. Response: {request.text}")
            return {}

# Example usage
API_KEY = 'RGAPI-e7a0cc73-2a12-4b8e-b909-71c950ac8132'  # Replace with your valid API key
#game_name_done = ['white space', 'Sheiden', 'Sushee', 'Afflictive', 'Zven']
#tag_line_done = ['srtty', '0001', 'NA1', 'NA1', 'KEKW1']
game_name = ['Zven', 'Dongdanny', 'wxtonytoppwd']
tag_line = ['KEKW1', 'pwdd', 'NA1']

for i in range(len(game_name)):
    ra = RiotApi(API_KEY)

    # Step 1: Get PUUID by Riot ID
    account_data = ra.get_puuid_by_riot_id(game_name[i], tag_line[i])
    puuid = account_data.get('puuid')
    print(f"{i}.")
    print(f"PUUID for {game_name[i]}#{tag_line[i]}: {puuid}")

    time.sleep(1.2)
    # Step 2: Get Summoner Data by PUUID
    if puuid:
        summoner_data = ra.get_summoner_by_puuid(puuid)
        print(f"Summoner data for : {summoner_data}")
        time.sleep(1.2)

        match_ids = ra.get_match_ids(puuid)
        print(f"Match ids length : {len(match_ids)}, data : {match_ids}")
        time.sleep(1.2)

        for match_id in match_ids:
            time.sleep(0.25)
            details = ra.match_details(match_id)
            if details['info']['queueId'] in {420, 430, 440, 400}:
                all_match_ids.append(match_id)
            time.sleep(1.2)
    else:
        print("PUUID not found, cannot retrieve summoner data.")


In [3]:
print(all_match_ids)
print(len(all_match_ids))

['NA1_5023056134', 'NA1_5023031055', 'NA1_5023007712', 'NA1_5022989394', 'NA1_5022892305', 'NA1_5022845019', 'NA1_5022803692', 'NA1_5022615462', 'NA1_5022367926', 'NA1_5022347567', 'NA1_5022338890', 'NA1_5022316593', 'NA1_5022295135', 'NA1_5022258504', 'NA1_5022231858', 'NA1_5022153255', 'NA1_5022115763', 'NA1_5022079826', 'NA1_5022044242', 'NA1_5021661507', 'NA1_5021643435', 'NA1_5021621581', 'NA1_5021601770', 'NA1_5021563574', 'NA1_5021479874', 'NA1_5021447997', 'NA1_5021413243', 'NA1_5021364555', 'NA1_5021250485', 'NA1_5021208663', 'NA1_5021186684', 'NA1_5020915710', 'NA1_5020900020', 'NA1_5020874686', 'NA1_5020844795', 'NA1_5020789680', 'NA1_5020743397', 'NA1_5020166824', 'NA1_5020141116', 'NA1_5020109706', 'NA1_5020079874', 'NA1_5020041575', 'NA1_5020006662', 'NA1_5019984014', 'NA1_5019446604', 'NA1_5019432781', 'NA1_5019408414', 'NA1_5019306176', 'NA1_5019270538', 'NA1_5019248270', 'NA1_5018773941', 'NA1_5018747201', 'NA1_5018727178', 'NA1_5018701467', 'NA1_5018661706', 'NA1_5018

In [4]:
import pickle
with open('list_of_match_ids_for_training.pkl', 'wb') as file:
    pickle.dump(all_match_ids, file)

In [90]:
# retieving all the details like avg kills, asssists of a player.
import requests

class RiotApi(object):
    def __init__(self, api_key: str, region="americas"):
        self.RIOT_API_KEY = api_key
        self.HEADER = {'X-Riot-Token': self.RIOT_API_KEY}
        self.REGION = region

    def get_match_history(self, puuid):
        url = f'https://{self.REGION}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids'
        response = requests.get(url, headers=self.HEADER)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: Received status code {response.status_code} for summoner request. Response: {response.text}")
            return []

    def get_match_data(self, match_id):
        url = f'https://{self.REGION}.api.riotgames.com/lol/match/v5/matches/{match_id}'
        response = requests.get(url, headers=self.HEADER)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: Received status code {response.status_code} for summoner request. Response: {response.text}")
            return {}

    def get_player_stats(self, match_data, puuid, champion_to_be_used):
        kills = 0
        deaths = 0
        assists = 0
        total_matches = 0
        champ_lvl = 0
        champ_exp = 0

        for player in match_data['info']['participants']:
            if player['puuid'] == puuid:
                kills += player['kills']
                deaths += player['deaths']
                assists += player['assists']
                total_matches += 1
                if player['championName'] == champion_to_be_used:
                    champ_lvl = player['champLevel']
                    champ_exp = player['champExperience']
        
        return kills, deaths, assists, total_matches, champ_lvl, champ_exp

    def get_summoner_by_puuid(self, puuid, country):
        self.REGION = country
        url = f"https://{self.REGION}.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}"
        request = requests.get(url, headers=self.HEADER)
        if request.status_code == 200:
            return request.json()
        else:
            print(f"Error: Received status code {request.status_code} for summoner request. Response: {request.text}")
            return {}
    
    def get_ranked_data(self, summoner_id, country):
        self.REGION = country
        url = f'https://{self.REGION}.api.riotgames.com/lol/league/v4/entries/by-summoner/{summoner_id}'
        response = requests.get(url, headers=self.HEADER)
        if response.status_code == 200:
            ranked_data = response.json()
            if ranked_data:
                return ranked_data[0]['tier'], ranked_data[0]['rank']
        print(f"Error: Received status code {response.status_code} for ranked data request. Response: {response.text}")
        return None, None

    def get_average_stats(self, puuid, champion_to_be_used):
        match_ids = self.get_match_history(puuid)
        total_kills = 0
        total_deaths = 0
        total_assists = 0
        total_matches = 0

        for match_id in match_ids:
            match_data = self.get_match_data(match_id)
            if match_data['info']['queueId'] not in {420, 430, 440, 400}:
                continue

            if match_data:
                country = match_id.split('_')[0]
                kills, deaths, assists, match_count, champ_lvl, champ_exp = self.get_player_stats(match_data, puuid, champion_to_be_used)
                total_kills += kills
                total_deaths += deaths
                total_assists += assists
                total_matches += match_count

        if total_matches > 0:
            average_kills = total_kills / total_matches
            average_deaths = total_deaths / total_matches
            average_assists = total_assists / total_matches
        else:
            average_kills = average_deaths = average_assists = 0
        
        summoner_data = self.get_summoner_by_puuid(puuid, country)
        summoner_id = summoner_data['id']
        tier, rank = self.get_ranked_data(summoner_id, country)

        return average_kills, average_deaths, average_assists, tier, rank, champ_lvl, champ_exp


# Example usage
API_KEY = 'RGAPI-70e702d9-1eb0-41e1-a096-fadff6804f51'
puuid = 'gPRDXczDal0ahDXchGTmtCt2Lj2PtW9qNwEkDuRAGg1nhNJDi4_pdCMminAN9idPpGiS1TzuzELLmg'
riot_api = RiotApi(API_KEY)
champion_to_be_used = 'Amumu'
average_kills, average_deaths, average_assists, tier, rank, champ_lvl, champ_exp = riot_api.get_average_stats(puuid, champion_to_be_used)
print("Average Kills:", average_kills)
print("Average Deaths:", average_deaths)
print("Average Assists:", average_assists)
print("Tier:", tier)
print("Rank:", rank)
print("Level of champion to be used in the prediction match:", champ_lvl)
print("Experience of champion to be used in the prediction match:", champ_exp)

Average Kills: 7.071428571428571
Average Deaths: 3.5714285714285716
Average Assists: 5.714285714285714
Tier: CHALLENGER
Rank: I
Level of champion to be used in the prediction match: 0
Experience of champion to be used in the prediction match: 0
